In [ ]:
from bs4 import BeautifulSoup
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests as rq
import seaborn as sns
import re
sns.set_context('notebook')

def getParamsRequestValue(i_reportmodel_num, i_reportmodel_xsl_num, section, year, semester): 
    r = rq.get('http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter', params={'ww_i_reportmodel' : i_reportmodel_num})
    parameters_map = { 'ww_x_UNITE_ACAD' : section, 'ww_x_PERIODE_ACAD' : year, 'ww_x_PERIODE_PEDAGO' : semester }
    
    soup = BeautifulSoup(r.text, 'html.parser')
    
    ps = {key: next((option.attrs['value'] for option in soup.find('select', {'name': key }) if option.text==value), None) for key, value in parameters_map.items()}
    o = {'ww_i_reportmodel' : i_reportmodel_num, 'ww_b_list' : 1, 'ww_i_reportModelXsl' : i_reportmodel_xsl_num, 'ww_x_HIVERETE' : 'null', 'dummy' : 'ok'}
    d = {**o, **ps}
    r = rq.get('http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter', params=d)
    soup = BeautifulSoup(r.text, 'html.parser')
    for p in soup.find_all('a'):
        if(p.text==', '.join([section, year, semester])):
            arr = re.search('\(\'(.+?)\'\)', p['onclick']).group(1).split("=")
            d[arr[0]] = int(arr[1])
                  
    return d

ww_i_reportmodel_num=133685247
ww_i_reportModelXsl_num=133685270
section = 'Informatique'
master_semesters = ['Master semestre 1', 'Master semestre 2', 'Master semestre 3', 'Master semestre 4']
years_semesters = {'2007-2008': master_semesters[0:2], 
                   '2008-2009': master_semesters, 
                   '2009-2010': master_semesters, 
                   '2010-2011': master_semesters, 
                   '2011-2012': master_semesters, 
                   '2012-2013': master_semesters, 
                   '2013-2014': master_semesters,
                   '2014-2015': master_semesters,
                   '2015-2016': master_semesters, 
                   '2016-2017': master_semesters }

year = '2015-2016'
semester = master_semesters[0]
payload = getParamsRequestValue(ww_i_reportmodel_num, ww_i_reportModelXsl_num, section, year, semester)
r = rq.get('http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html?', params=payload)
soup = BeautifulSoup(r.text, 'html.parser')
list_attr = []
attributs = soup.find_all('th')
for attr in attributs:
    if(attr.string!=None):
        list_attr.append(attr.string)

list_attr.append("")

df_all = pd.DataFrame(columns=list_attr)
for year,semesters in years_semesters.items():
    for semester in semesters:
        payload = getParamsRequestValue(ww_i_reportmodel_num, ww_i_reportModelXsl_num, section, year, semester)
        r = rq.get('http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html?', params=payload)
        soup = BeautifulSoup(r.text, 'html.parser')
        tds = soup.find_all('td')
        df = pd.DataFrame(columns=list_attr)
        pers = pd.Series()
        i=0
        j=0
        for elem in tds:
            pers.set_value(list_attr[(i%len(list_attr))], elem.string)
            i+=1
            if(i%len(list_attr)==0):
                j+=1
                df.loc[j]=pers
                pers.iloc[1:]

        # We delete the empty column (and others if needed)
        del df[""]
        print(semester)
        df["Master Semestre"]=re.findall(r'\d',semester)[0]
        df["Year"]=year
        df_all = df_all.append(df, ignore_index=True)
        
df_all.to_csv('student_master.csv', sep=';', encoding='utf-8')
        

In [ ]:
students_master=df_all[['No Sciper','Civilité', 'Nom Prénom', 'Master Semestre', 'Statut', 'Spécialisation', 'Mineur', 'Year']]
unique_sciper=pd.unique(students_master['No Sciper'])
unique_sciper_df=pd.DataFrame(unique_sciper, columns=['No Sciper'])

k=0
semesters=[]
nb_semesters=[]
years=[]
civilites=[]
statuts=[]
mineurs=[]
specialisations=[]

students_master=students_master.sort_values(by=['Year','Master Semestre'])

for i in range(0,len(unique_sciper_df)):
    student=students_master[students_master['No Sciper']==unique_sciper_df['No Sciper'][i]]
    semester=[]
    year=[]
    statut=[]
    specialisation=[]
    mineur=[]
    civilites.append(student['Civilité'].values[0])
    nb_semester=0
    for student_semester in student['Master Semestre']:
        semester.append(student_semester)
        nb_semester+=1
    for student_year in student['Year']:
        year.append(student_year)
    for student_statut in student['Statut']:
        statut.append(student_statut)
    for student_spe in student['Spécialisation']:
        specialisation.append(student_spe)
    for student_mineur in student['Mineur']:
        mineur.append(student_mineur)
    semesters.append(semester)
    years.append(year)
    nb_semesters.append(nb_semester)
    statuts.append(statut)
    specialisations.append(specialisation)
    mineurs.append(mineur)
    
unique_sciper_df['Semestres']=semesters
unique_sciper_df['Nb semestres']=nb_semesters
unique_sciper_df['Année']=years
unique_sciper_df['Civilité']=civilites
unique_sciper_df['Statut']=statuts
unique_sciper_df['Spécialisation']=specialisations
unique_sciper_df['Mineur']=mineurs
unique_sciper_df  